In [127]:
import pandas as pd
import json
import numpy as np
import re

In [128]:
with open('/content/crawling_data/major_accreditation_data.json') as f:
  data = json.load(f)


In [129]:
arrayData = np.array(data['data'])
dataframe = pd.DataFrame(arrayData, columns=[
    'University_Name','Major_Name', 'Degree',
    'Code', 'SK_Num', 'SK_Year', 'Accreditation',
    'Expired_Date', 'Status'
])

In [130]:
dataframe = pd.DataFrame(arrayData, columns=['University_Name','Major_Name', 'Degree', 'Code', 'SK_Num', 'SK_Year', 'Accreditation', 'Expired_Date', 'Status'])

In [131]:
accreditations = ['A', 'Unggul']
region_codes = ['03','04','05','06','07','PTN']
degrees = ['S1']

accred_filtered_df = dataframe[dataframe['Accreditation'].isin(accreditations)]
region_filtered_df = accred_filtered_df[accred_filtered_df['Code'].isin(region_codes)]
degree_filtered_df = region_filtered_df[region_filtered_df['Degree'].isin(degrees)]

new_df = degree_filtered_df[["University_Name", "Major_Name", "Code", "Accreditation"]]

new_df["University_Name"] = new_df["University_Name"].map(str.title)
new_df["Major_Name"] = new_df["Major_Name"].map(str.title)

<ipython-input-131-feae97049c88>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["University_Name"] = new_df["University_Name"].map(str.title)
<ipython-input-131-feae97049c88>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Major_Name"] = new_df["Major_Name"].map(str.title)


In [132]:
major_mapping_data = pd.read_excel('/content/crawling_data/duplicates_major_mapping.xlsx')
new_df = pd.merge(new_df, major_mapping_data, on='Major_Name', how='left')
new_df['temp_major_name'] = new_df['New_Major_Name'].combine_first(new_df['Major_Name'])

In [133]:
new_df = new_df.drop(columns=['Major_Name', 'New_Major_Name'])
new_df.rename(columns={"temp_major_name": "Major_Name",}, inplace=True)
new_df

,University_Name,Code,Accreditation,Major_Name
0,Universitas Katolik Indonesia Atma Jaya,03,A,Ekonomi Pembangunan
1,Universitas Muhammadiyah Magelang,06,A,Pendidikan Guru Madrasah Ibtidaiyah
2,Universitas Sebelas Maret,PTN,A,Pendidikan Kimia
3,Universitas Udayana,PTN,A,Industri Perjalanan Wisata
4,Universitas Hasanuddin,PTN,A,Sastra Inggris
...,...,...,...,...
2369,Institut Bisnis Dan Informatika Kwik Kian Gie,03,Unggul,Manajemen
2370,Institut Pertanian Bogor,PTN,Unggul,Ekonomi Pembangunan
2371,Universitas Stikubank,06,Unggul,Manajemen
2372,Universitas Muhammadiyah Jakarta,03,Unggul,Ilmu Ekonomi Islam


In [134]:
univ_mapping_data = pd.read_excel('/content/crawling_data/duplicates_university_mapping.xlsx')
new_df = pd.merge(new_df, univ_mapping_data, on='University_Name', how='left')
new_df['temp_university_name'] = new_df['New_University_Name'].combine_first(new_df['University_Name'])

In [135]:
new_df = new_df.drop(columns=['University_Name', 'New_University_Name'])
new_df.rename(columns={"temp_university_name": "University_Name",}, inplace=True)
new_df

,Code,Accreditation,Major_Name,University_Name
0,03,A,Ekonomi Pembangunan,Universitas Katolik Indonesia Atma Jaya
1,06,A,Pendidikan Guru Madrasah Ibtidaiyah,Universitas Muhammadiyah Magelang
2,PTN,A,Pendidikan Kimia,Universitas Sebelas Maret
3,PTN,A,Industri Perjalanan Wisata,Universitas Udayana
4,PTN,A,Sastra Inggris,Universitas Hasanuddin
...,...,...,...,...
2371,03,Unggul,Manajemen,Institut Bisnis Dan Informatika Kwik Kian Gie
2372,PTN,Unggul,Ekonomi Pembangunan,Institut Pertanian Bogor
2373,06,Unggul,Manajemen,Universitas Stikubank
2374,03,Unggul,Ilmu Ekonomi Islam,Universitas Muhammadiyah Jakarta


In [136]:
def region_name(code):
    if code == '03':
        return 'DKI Jakarta'
    elif code == '04':
        return 'Jawa Barat / Banten'
    elif code == '05':
        return 'DI Yogyakarta'
    elif code == '06':
        return 'Jawa Tengah'
    elif code == '07':
        return 'Jawa Timur'
    else:
        return ''

new_df['Region'] = new_df['Code'].apply(region_name)
new_df['Type'] = np.where(new_df['Code'] == 'PTN', 'PTN', 'PTS')

In [137]:
new_df.shape

(2376, 6)

In [138]:
new_df = new_df.dropna(how='any',axis=0)
new_df.drop_duplicates

<bound method DataFrame.drop_duplicates of      Code Accreditation                           Major_Name  \
0      03             A                  Ekonomi Pembangunan   
1      06             A  Pendidikan Guru Madrasah Ibtidaiyah   
2     PTN             A                     Pendidikan Kimia   
3     PTN             A           Industri Perjalanan Wisata   
4     PTN             A                       Sastra Inggris   
...   ...           ...                                  ...   
2371   03        Unggul                            Manajemen   
2372  PTN        Unggul                  Ekonomi Pembangunan   
2373   06        Unggul                            Manajemen   
2374   03        Unggul                   Ilmu Ekonomi Islam   
2375  PTN        Unggul                       Sastra Inggris   

                                    University_Name       Region Type  
0           Universitas Katolik Indonesia Atma Jaya  DKI Jakarta  PTS  
1                 Universitas Muhammadiyah Magelang  Jawa Tengah  PTS  
2                         Universitas Sebelas Maret               PTN  
3                               Universitas Udayana               PTN  
4                            Universitas Hasanuddin               PTN  
...                                             ...          ...  ...  
2371  Institut Bisnis Dan Informatika Kwik Kian Gie  DKI Jakarta  PTS  
2372                       Institut Pertanian Bogor               PTN  
2373                          Universitas Stikubank  Jawa Tengah  PTS  
2374               Universitas Muhammadiyah Jakarta  DKI Jakarta  PTS  
2375                            Universitas Samudra               PTN  

[2376 rows x 6 columns]>

In [139]:
new_df.shape

(2376, 6)

In [140]:
new_df.head()

,Code,Accreditation,Major_Name,University_Name,Region,Type
0,03,A,Ekonomi Pembangunan,Universitas Katolik Indonesia Atma Jaya,DKI Jakarta,PTS
1,06,A,Pendidikan Guru Madrasah Ibtidaiyah,Universitas Muhammadiyah Magelang,Jawa Tengah,PTS
2,PTN,A,Pendidikan Kimia,Universitas Sebelas Maret,,PTN
3,PTN,A,Industri Perjalanan Wisata,Universitas Udayana,,PTN
4,PTN,A,Sastra Inggris,Universitas Hasanuddin,,PTN


In [141]:
PTN_df = new_df.loc[(new_df['Type'] == 'PTN')]
PTN_df = PTN_df.drop(['Code','Region'], axis=1)
PTN_df

,Accreditation,Major_Name,University_Name,Type
2,A,Pendidikan Kimia,Universitas Sebelas Maret,PTN
3,A,Industri Perjalanan Wisata,Universitas Udayana,PTN
4,A,Sastra Inggris,Universitas Hasanuddin,PTN
10,A,Sosiologi,Universitas Sam Ratulangi,PTN
13,A,Ekonomi Pertanian Dan Agribisnis,Universitas Gadjah Mada,PTN
...,...,...,...,...
2364,Unggul,Pendidikan Ilmu Pengetahuan Sosial,Universitas Pendidikan Indonesia,PTN
2367,Unggul,Pendidikan Fisika,Universitas Lambung Mangkurat,PTN
2369,Unggul,Teknologi Pangan,Universitas Pembangunan Nasional Veteran Jawa ...,PTN
2372,Unggul,Ekonomi Pembangunan,Institut Pertanian Bogor,PTN


In [142]:
patterns = [
    (r'\d+', ''),         # Remove numbers
    (r'&Quot;', ''),      # Remove "&Quot;"
    (r'\([^()]*\)', ''),      # Remove Anything between "(" and ")" along with both "(" and ")"
    ('(,).*',''),         # Remove everything after a comma
    ("-"," "),            # Remove dash
    ("\.", ""),           # Remove period
    (r'^\s+', ''),        # Remove leading space
    (r'\s+', ' ')         # Replace multiple spaces with one space
]
for pattern, replacement in patterns:
    PTN_df['University_Name'] = PTN_df['University_Name'].apply(lambda x: re.sub(pattern, replacement, x)).map(str.title)

PTN_df['University_Name'].unique()

array(['Universitas Sebelas Maret', 'Universitas Udayana',
       'Universitas Hasanuddin', 'Universitas Sam Ratulangi',
       'Universitas Gadjah Mada', 'Universitas Lampung',
       'Universitas Airlangga', 'Universitas Syiah Kuala',
       'Universitas Diponegoro', 'Universitas Padjadjaran',
       'Universitas Pendidikan Indonesia', 'Universitas Pattimura',
       'Universitas Negeri Gorontalo', 'Universitas Halu Oleo',
       'Universitas Lambung Mangkurat', 'Universitas Brawijaya',
       'Universitas Indonesia', 'Universitas Tanjungpura',
       'Universitas Riau', 'Universitas Negeri Malang',
       'Universitas Negeri Semarang', 'Universitas Pendidikan Ganesha',
       'Universitas Pembangunan Nasional Veteran Yogyakarta',
       'Universitas Sriwijaya', 'Universitas Andalas',
       'Universitas Jenderal Soedirman', 'Institut Pertanian Bogor',
       'Universitas Khairun', 'Universitas Negeri Yogyakarta',
       'Universitas Palangka Raya', 'Universitas Negeri Jakarta',
    

In [143]:
PTN_mapping_data = pd.read_excel('/content/crawling_data/mapping_region_code.xlsx')
PTN_mapping_data = PTN_mapping_data.astype({'Region_Code':'string'})
PTN_mapping_data['Region_Code'] = PTN_mapping_data['Region_Code'].str.zfill(2)
PTN_mapping_data.rename(columns={"Region_Code": "Code"}, inplace = True)
PTN_data = pd.merge(PTN_df, PTN_mapping_data, on='University_Name')
PTN_data

,Accreditation,Major_Name,University_Name,Type,Code,Region
0,A,Pendidikan Kimia,Universitas Sebelas Maret,PTN,06,Jawa Tengah
1,A,Pendidikan Geografi,Universitas Sebelas Maret,PTN,06,Jawa Tengah
2,A,Pendidikan Seni Rupa,Universitas Sebelas Maret,PTN,06,Jawa Tengah
3,A,Pendidikan Matematika,Universitas Sebelas Maret,PTN,06,Jawa Tengah
4,A,Pendidikan Matematika,Universitas Sebelas Maret,PTN,06,Jawa Tengah
...,...,...,...,...,...,...
694,A,Agroekoteknologi,Universitas Trunojoyo,PTN,07,Jawa Timur
695,Unggul,Sastra Indonesia,Universitas Trunojoyo,PTN,07,Jawa Timur
696,Unggul,Pendidikan Guru Pendidikan Anak Usia Dini,Universitas Trunojoyo,PTN,07,Jawa Timur
697,Unggul,Agribisnis,Universitas Trunojoyo,PTN,07,Jawa Timur


In [144]:
print(PTN_mapping_data['Code'].dtypes)

string


In [145]:
PTN_data = pd.merge(PTN_df, PTN_mapping_data, on='University_Name')
PTN_data

,Accreditation,Major_Name,University_Name,Type,Code,Region
0,A,Pendidikan Kimia,Universitas Sebelas Maret,PTN,06,Jawa Tengah
1,A,Pendidikan Geografi,Universitas Sebelas Maret,PTN,06,Jawa Tengah
2,A,Pendidikan Seni Rupa,Universitas Sebelas Maret,PTN,06,Jawa Tengah
3,A,Pendidikan Matematika,Universitas Sebelas Maret,PTN,06,Jawa Tengah
4,A,Pendidikan Matematika,Universitas Sebelas Maret,PTN,06,Jawa Tengah
...,...,...,...,...,...,...
694,A,Agroekoteknologi,Universitas Trunojoyo,PTN,07,Jawa Timur
695,Unggul,Sastra Indonesia,Universitas Trunojoyo,PTN,07,Jawa Timur
696,Unggul,Pendidikan Guru Pendidikan Anak Usia Dini,Universitas Trunojoyo,PTN,07,Jawa Timur
697,Unggul,Agribisnis,Universitas Trunojoyo,PTN,07,Jawa Timur


In [146]:
PTS_data = new_df.loc[(new_df['Type'] == 'PTS')]
PTS_data

,Code,Accreditation,Major_Name,University_Name,Region,Type
0,03,A,Ekonomi Pembangunan,Universitas Katolik Indonesia Atma Jaya,DKI Jakarta,PTS
1,06,A,Pendidikan Guru Madrasah Ibtidaiyah,Universitas Muhammadiyah Magelang,Jawa Tengah,PTS
5,04,A,Desain Komunikasi Visual,Universitas Wanita Internasional,Jawa Barat / Banten,PTS
6,03,A,Desain Komunikasi Visual,Universitas Persada Indonesia Yai,DKI Jakarta,PTS
7,07,A,Teknik Elektro,Universitas Surabaya,Jawa Timur,PTS
...,...,...,...,...,...,...
2368,06,Unggul,Pendidikan Guru Sekolah Dasar,Universitas PGRI Semarang,Jawa Tengah,PTS
2370,07,Unggul,Manajemen,Universitas Katolik Widya Mandala Surabaya,Jawa Timur,PTS
2371,03,Unggul,Manajemen,Institut Bisnis Dan Informatika Kwik Kian Gie,DKI Jakarta,PTS
2373,06,Unggul,Manajemen,Universitas Stikubank,Jawa Tengah,PTS


In [147]:
for pattern, replacement in patterns:
    PTS_data['University_Name'] = PTS_data['University_Name'].apply(lambda x: re.sub(pattern, replacement, x)).map(str.title)

PTS_data['University_Name'].unique()

<ipython-input-147-433156faebb7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PTS_data['University_Name'] = PTS_data['University_Name'].apply(lambda x: re.sub(pattern, replacement, x)).map(str.title)
<ipython-input-147-433156faebb7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PTS_data['University_Name'] = PTS_data['University_Name'].apply(lambda x: re.sub(pattern, replacement, x)).map(str.title)
<ipython-input-147-433156faebb7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

array(['Universitas Katolik Indonesia Atma Jaya',
       'Universitas Muhammadiyah Magelang',
       'Universitas Wanita Internasional',
       'Universitas Persada Indonesia Yai', 'Universitas Surabaya',
       'Institut Teknologi Nasional Malang', 'Universitas Islam Bandung',
       'Universitas Kuningan',
       'Sekolah Tinggi Ilmu Ekonomi Indonesia Jakarta',
       'Universitas Sains Alqur An', 'Universitas Muhammadiyah Tangerang',
       'Universitas Presiden', 'Universitas Mercu Buana Yogyakarta',
       'Institut Bisnis Dan Informatika Kwik Kian Gie',
       'Universitas Muhammadiyah Malang',
       'Universitas Muhammadiyah Jember', 'Universitas Muria Kudus',
       'Universitas Islam Sultan Agung',
       'Universitas Kristen Krida Wacana',
       'Universitas Prof Dr Moestopo ', 'Universitas Tarumanagara',
       'Universitas Indraprasta Pgri', 'Universitas Kristen Satya Wacana',
       'Universitas Bina Nusantara', 'Universitas Ibn Khaldun',
       'Universitas Muhammadiyah

In [148]:
major_accreditation = pd.concat([PTS_data,PTN_data]).reset_index(drop=True)
major_accreditation.rename(columns={
    "University_Name": "university_name",
    "Major_Name": "major_name",
    "Code": "code",
    "Accreditation": "accreditation",
    "Region": "region_name",
    "Type": "type"
}, inplace = True)
major_accreditation

,code,accreditation,major_name,university_name,region_name,type
0,03,A,Ekonomi Pembangunan,Universitas Katolik Indonesia Atma Jaya,DKI Jakarta,PTS
1,06,A,Pendidikan Guru Madrasah Ibtidaiyah,Universitas Muhammadiyah Magelang,Jawa Tengah,PTS
2,04,A,Desain Komunikasi Visual,Universitas Wanita Internasional,Jawa Barat / Banten,PTS
3,03,A,Desain Komunikasi Visual,Universitas Persada Indonesia Yai,DKI Jakarta,PTS
4,07,A,Teknik Elektro,Universitas Surabaya,Jawa Timur,PTS
...,...,...,...,...,...,...
1498,07,A,Agroekoteknologi,Universitas Trunojoyo,Jawa Timur,PTN
1499,07,Unggul,Sastra Indonesia,Universitas Trunojoyo,Jawa Timur,PTN
1500,07,Unggul,Pendidikan Guru Pendidikan Anak Usia Dini,Universitas Trunojoyo,Jawa Timur,PTN
1501,07,Unggul,Agribisnis,Universitas Trunojoyo,Jawa Timur,PTN


In [149]:
unique_university = major_accreditation[[
    'university_name', 'type', 'code'
  ]].drop_duplicates()
unique_university.sort_values(by='university_name', inplace=True)
unique_university['id'] = range(1, len(unique_university)+1)
unique_university = unique_university.reindex(columns=[
    'id', 'university_name',
    'type', 'code'
  ]).reset_index(drop=True)
unique_university.rename(columns={
    "code": "region_code"
  }, inplace = True)
unique_university

,id,university_name,type,region_code
0,1,Institut Bio Scientia Internasional Indonesia,PTS,03
1,2,Institut Bisnis Dan Informatika Kesatuan,PTS,04
2,3,Institut Bisnis Dan Informatika Kwik Kian Gie,PTS,03
3,4,Institut Ilmu Sosial Dan Manajemen Stiami,PTS,03
4,5,Institut Kesenian Jakarta Lpkj,PTS,03
...,...,...,...,...
183,184,Universitas Widyatama,PTS,04
184,185,Universitas Wijaya Kusuma Surabaya,PTS,07
185,186,Universitas Wijaya Putra,PTS,07
186,187,Universitas Wiraraja,PTS,07


In [150]:
unique_major = major_accreditation[['major_name']].drop_duplicates()
unique_major.sort_values(by='major_name', inplace=True)
unique_major['id'] = range(1, len(unique_major)+1)
unique_major = unique_major.reindex(columns=[
    'id', 'major_name'
  ]).reset_index(drop=True)
unique_major

,id,major_name
0,1,Agribisnis
1,2,Agrobisnis Perikanan
2,3,Agroekoteknologi
3,4,Agronomi Dan Hortikultura
4,5,Agroteknologi
...,...,...
252,253,Teknologi Pangan
253,254,Teknologi Pasca Panen
254,255,Teknologi Pendidikan
255,256,Teknologi Produksi Ternak


In [151]:
unique_region = major_accreditation[[
    'region_name', 'code'
  ]].drop_duplicates()
unique_region = unique_region.reindex(columns=[
    'code', 'region_name'
  ]).sort_values(by='code').reset_index(drop=True)
unique_region

,code,region_name
0,03,DKI Jakarta
1,04,Jawa Barat / Banten
2,05,DI Yogyakarta
3,06,Jawa Tengah
4,07,Jawa Timur


In [152]:
unique_major_accreditation = pd.merge(
    major_accreditation, unique_university,
  on='university_name')
unique_major_accreditation = pd.merge(
    unique_major_accreditation, unique_major,
  on='major_name')
unique_major_accreditation = unique_major_accreditation[[
    'id_x', 'id_y', 'accreditation'
  ]]
unique_major_accreditation.rename(columns={
    "id_x": "university_id",
    "id_y": "major_id"
  }, inplace = True)
unique_major_accreditation = unique_major_accreditation.drop_duplicates(
    subset=['university_id', 'major_id']
  )
unique_major_accreditation

,university_id,major_id,accreditation
0,89,29,A
1,169,29,A
2,76,29,Unggul
3,113,29,A
4,100,29,A
...,...,...,...
1513,167,134,Unggul
1514,167,170,Unggul
1515,167,148,Unggul
1516,133,144,Unggul


In [153]:
#sql insert

def SQL_INSERT(SOURCE, TARGET):
    sql_texts = 'INSERT INTO '+ TARGET +' ('+ str(', '.join(SOURCE.columns))+ ') VALUES \n'
    for index, row in SOURCE.iterrows():
        sql_texts += (str(tuple(row.values)) + ", \n")
    sql_texts = sql_texts[:-3]
    return sql_texts

In [154]:
univ_insert_sql = SQL_INSERT(unique_university, 'universities')
major_insert_sql = SQL_INSERT(unique_major, 'majors')
region_insert_sql = SQL_INSERT(unique_region, 'regions')
accreditation_insert_sql = SQL_INSERT(unique_major_accreditation, 'university_major_accreditations')

In [155]:
print(region_insert_sql)

INSERT INTO regions (code, region_name) VALUES 
('03', 'DKI Jakarta'), 
('04', 'Jawa Barat / Banten'), 
('05', 'DI Yogyakarta'), 
('06', 'Jawa Tengah'), 
('07', 'Jawa Timur')


In [156]:
f= open("insert_seed_region.sql","w+")
f.write(region_insert_sql)

f= open("insert_seed_university.sql","w+")
f.write(univ_insert_sql)

f= open("insert_seed_major.sql","w+")
f.write(major_insert_sql)

f= open("insert_seed_accreditation.sql","w+")
f.write(accreditation_insert_sql)

f.close()

In [157]:
unique_major

,id,major_name
0,1,Agribisnis
1,2,Agrobisnis Perikanan
2,3,Agroekoteknologi
3,4,Agronomi Dan Hortikultura
4,5,Agroteknologi
...,...,...
252,253,Teknologi Pangan
253,254,Teknologi Pasca Panen
254,255,Teknologi Pendidikan
255,256,Teknologi Produksi Ternak
